<a href="https://colab.research.google.com/github/hadywalied/DistillPegasus/blob/main/xsum_pegasus_student_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
! pip install datasets transformers rouge-score nltk wandb

In [19]:
!git clone https://github.com/huggingface/transformers
%cd transformers
!pip install .

%  cd examples
!pip install -r /content/transformers/examples/pytorch/summarization/requirements.txt
% cd ..
% cd ..

fatal: destination path 'transformers' already exists and is not an empty directory.
/content/transformers
Processing /content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.7.0.dev0-cp37-none-any.whl size=2351635 sha256=4563a5e80bc79da63a5dda84ade92ffd2459782736c3d6660ec2916a158df0bc
  Stored in directory: /tmp/pip-ephem-wheel-cache-rehrqqwz/wheels/23/19/dd/2561a4e47240cf6b307729d58e56f8077dd0c698f5992216cf
Successfully built transformers
  Found existing installation: transformers 4.7.0.dev0
    Uninstalling transformers-4.7.0.dev0:
      Successfully uninstalled transformers-4.7.0.dev0


/content/transformers/examples
/content/transformers
/content


In [20]:
!CUDA_LAUNCH_BLOCKING=1

In [21]:
student_number = 2
data_name = 'xsum'
#model_name = f'google/pegasus-{data_name}'
model_name = 'tuner007/pegasus_paraphrase'

In [22]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset(data_name)
metric = load_metric("rouge")

Using custom data configuration default
Reusing dataset xsum (/root/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499)


In [23]:
from google.colab import drive
import os

drive.mount('/content/drive')

#directory = f'/content/drive/MyDrive/GP/dataset'
#if not os.path.exists(directory):
#    os.makedirs(directory)

PATH = f'/content/drive/MyDrive/GP/student_{data_name}_{student_number}'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [25]:
summarization_name_mapping = {
    "amazon_reviews_multi": ("review_body", "review_title"),
    "big_patent": ("description", "abstract"),
    "cnn_dailymail": ("article", "highlights"),
    "orange_sum": ("text", "summary"),
    "pn_summary": ("article", "summary"),
    "psc": ("extract_text", "summary_text"),
    "samsum": ("dialogue", "summary"),
    "thaisum": ("body", "summary"),
    "xglue": ("news_body", "news_title"),
    "xsum": ("document", "summary"),
    "gigaword": ("document", "summary"),
    "wiki_summary": ("article", "highlights"),
}

In [26]:
max_input_length = 1024
max_target_length = 60

def preprocess_function(examples):
    inputs = [doc for doc in examples[summarization_name_mapping[data_name][0]] ]
    model_inputs = tokenizer(inputs, truncation=True, padding = 'max_length', max_length = max_input_length)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples[summarization_name_mapping[data_name][1]], truncation=True,padding = 'max_length', max_length = max_target_length)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [27]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

# Model Preparation

In [28]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, Trainer

teacher = AutoModelForSeq2SeqLM.from_pretrained(model_name, max_length = 1024,max_position_embeddings=1024)


In [29]:
#Student configuration

import warnings
import torch
from torch import nn
from typing import Optional, Tuple, List, Union
from transformers import PegasusModel, PegasusConfig, PegasusForConditionalGeneration
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, PreTrainedModel
from transformers import SummarizationPipeline
import gc

students_config_book = {
    '2': PegasusConfig(encoder_layers=2, decoder_layers=2),
    '4': PegasusConfig(encoder_layers=4, decoder_layers=4),
    '6': PegasusConfig(encoder_layers=6, decoder_layers=6),
    '8': PegasusConfig(encoder_layers=8, decoder_layers=8),
    '10': PegasusConfig(encoder_layers=10, decoder_layers=10),
    '12': PegasusConfig(encoder_layers=12, decoder_layers=12),
    '16': PegasusConfig(encoder_layers=16, decoder_layers=16)
}


LAYERS_TO_COPY = {   
    4:{
        1: [0],
        2: [0, 3],
        3: [0, 1, 3],
        4: [0, 1, 2, 3],
    },
    8:{
        1: [0],
        2: [0, 7],
        3: [0, 4, 7],
        4: [0, 3, 6, 7],
        6: [0, 2, 3, 5, 6, 7],
        8: list(range(8)),  
    },    
    12: {
        1: [0],
        2: [0, 11],
        3: [0, 6, 11],
        4: [0, 4, 9, 11],
        6: [0, 2, 5, 8, 10, 11],
        8: [0, 1, 3, 5, 7, 9, 10, 11],
        12: list(range(12)),  
    },
    16: {  # maps  num layers in student -> which teacher layers to copy
        1: [0],
        2: [0, 15],
        3: [0, 8, 15],
        4: [0, 5, 10, 15],
        6: [0, 3, 6, 9, 12, 15],
        8: [0, 2, 4, 6, 8, 10, 12, 15],
        9: [0, 1, 3, 5, 7, 9, 11, 13, 15],
        12: [0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 13, 15],
        16: list(range(16)),
    },}
LAYERS_TO_SUPERVISE = {
    # maps  num layers in student -> which teacher layers to copy.
    8: {1: [5], 2: [3, 5], 3: [1, 4, 5], 4: [1, 2, 4, 5]},
    12: {1: [11], 2: [5, 11], 3: [3, 7, 11], 4:[1, 3, 7, 11],6: [1, 3, 5, 8, 10, 11], 8:[1,2,3,5,7,8,9,11] },
    16: {1: [15], 4: [4, 9, 12, 15], 8: [1, 3, 5, 7, 9, 11, 13, 15], 12:[1,2,3,5,7,8,9,11,12,13,14,15]},
}


def copy_layers(src_layers: nn.ModuleList, dest_layers: nn.ModuleList, layers_to_copy) -> None:
    layers_to_copy = nn.ModuleList([src_layers[i] for i in layers_to_copy])
    assert len(dest_layers) == len(
        layers_to_copy), f"{len(dest_layers)} != {len(layers_to_copy)}"
    dest_layers.load_state_dict(layers_to_copy.state_dict())

# Copied from transformers.models.bart.modeling_bart.shift_tokens_right


def shift_tokens_right(input_ids: torch.Tensor, pad_token_id: int, decoder_start_token_id: int):
    """
    Shift input ids one token to the right.
    """
    shifted_input_ids = input_ids.new_zeros(input_ids.shape)
    shifted_input_ids[:, 1:] = input_ids[:, :-1].clone()
    shifted_input_ids[:, 0] = decoder_start_token_id

    assert pad_token_id is not None, "self.model.config.pad_token_id has to be defined."
    # replace possible -100 values in labels by `pad_token_id`
    shifted_input_ids.masked_fill_(shifted_input_ids == -100, pad_token_id)

    return shifted_input_ids


def pick_layers_to_copy(n_student, n_teacher):
    try:
        val = LAYERS_TO_COPY[n_teacher][n_student]
        return val
    except KeyError:
        if n_student != n_teacher:
            warnings.warn(
                f"no hardcoded layers to copy for teacher {n_teacher} -> student {n_student}, defaulting to first {n_student}"
            )
        return list(range(n_student))

def get_layers_to_supervise(n_student, n_teacher) -> List[int]:
    """Used or the --supervise_forward kwarg"""
    if n_student > n_teacher:
        raise ValueError(f"Cannot perform intermediate supervision for student {n_student} > teacher {n_teacher}")
    elif n_teacher == n_student:
        return list(range(n_teacher))
    elif n_student == 1:
        return [n_teacher - 1]
    else:
        return LAYERS_TO_SUPERVISE[n_teacher][n_student]

def create_student_with_configuration(teacher,
                                      e=None,
                                      d=None,
                                      copy_first_teacher_layers = False,
                                      save_path='./student'):

    teacher.eval()
    teacher_e, teacher_d = teacher.config.encoder_layers, teacher.config.decoder_layers
    init_kwargs = teacher.config.to_diff_dict()
    if e is None:
        e = teacher_e
    if d is None:
        d = teacher_d
    init_kwargs.update({"encoder_layers": e, "decoder_layers": d})
    student_cfg = teacher.config_class(**init_kwargs)
    print(student_cfg.max_length)
    
    student = AutoModelForSeq2SeqLM.from_config(student_cfg)
    # Start by copying the full teacher state dict this will copy the first N teacher layers to the student.
    info = student.load_state_dict(teacher.state_dict(), strict=False)
    # every student key should have a teacher keys.
    assert info.missing_keys == [], info.missing_keys

    if copy_first_teacher_layers:  # Our copying is done. We just log and save
        e_layers_to_copy, d_layers_to_copy = list(range(e)), list(range(d))
        #student.save_pretrained(save_path)
        return student, e_layers_to_copy, d_layers_to_copy

    # Decide which layers of the teacher to copy. Not exactly alternating -- we try to keep first and last layer.
    e_layers_to_copy: List[int] = pick_layers_to_copy(e, teacher_e)
    d_layers_to_copy: List[int] = pick_layers_to_copy(d, teacher_d)

    copy_layers(teacher.model.encoder.layers,
                student.model.encoder.layers, e_layers_to_copy)
    copy_layers(teacher.model.decoder.layers,
                student.model.decoder.layers, d_layers_to_copy)

    student.config.init_metadata = dict(
        teacher_type=teacher.config.model_type,
        copied_encoder_layers=e_layers_to_copy,
        copied_decoder_layers=d_layers_to_copy,
    )
    #student.save_pretrained(save_path)
    # Save information about copying for easier reproducibility

    return student, e_layers_to_copy, d_layers_to_copy
#student = create_student_with_configuration(teacher,
#                                      e=4,
#                                      d=4,
#                                      copy_first_teacher_layers = False,
#                                      save_path='./student')
#import gc
#del copy_teacher
gc.collect()


1375

In [30]:
student, e_layers_list, d_layers_list = create_student_with_configuration(
                                      teacher,
                                      e=16,
                                      d=student_number,
                                      copy_first_teacher_layers = False,
                                      save_path=PATH)
 

del teacher
gc.collect()
torch.cuda.empty_cache()

1024


In [31]:
for param in student.model.encoder.parameters():
  param.requires_grad = False
for param in student.model.shared.parameters():
  param.requires_grad = False
for param in student.model.encoder.embed_tokens.parameters():
  param.requires_grad = False
for param in student.model.encoder.embed_positions.parameters():
  param.requires_grad = False
for param in student.model.decoder.embed_tokens.parameters():
  param.requires_grad = False
for param in student.model.decoder.embed_positions.parameters():
  param.requires_grad = False

# The Training Loop

In [32]:
import nltk
nltk.download('punkt')

import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [33]:

class MyTrainer(Seq2SeqTrainer):
    
    def shift_tokens_right(self, input_ids, pad_token_id):
      """Shift input ids one token to the right, and wrap the last non pad token (usually <eos>)."""
      prev_output_tokens = input_ids.clone()
      #print(pad_token_id, input_ids)
      x= (input_ids.ne(pad_token_id).sum(dim=1) - 1)
      index_of_eos = x.unsqueeze(-1)
      prev_output_tokens[:, 0] = input_ids.gather(1, index_of_eos).squeeze()
      prev_output_tokens[:, 1:] = input_ids[:, :-1]
      return prev_output_tokens
    
    
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs["labels"]
        pad_token_id = tokenizer.pad_token_id
        decoder_input_ids = self.shift_tokens_right(labels, pad_token_id)
        input_ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        #print(f"input: {input_ids}, masks: {attention_mask}")
        outputs = model(input_ids,
                        attention_mask=attention_mask,
                        decoder_input_ids=decoder_input_ids,
                        output_hidden_states=False,
                        output_attentions=False,
                        use_cache=False)
        logits = outputs["logits"]
        #print(f'logits: {logits}, labels: {labels}')
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, logits.shape[-1]),labels.view(-1))
        #print(loss)
        return (loss, outputs) if return_outputs else loss

In [34]:
batch_size = 16
args = Seq2SeqTrainingArguments(
    output_dir=f'{PATH}/output',
    #evaluation_strategy = "steps",
    #eval_steps = 500,
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps = 16,
    eval_accumulation_steps = 16,
    warmup_steps=500,
    weight_decay=0.01,
    #weight_decay=0,
    logging_dir=f'{PATH}/logs',            # directory for storing logs
    logging_steps=10,
    save_total_limit=2,
    num_train_epochs=4,
    predict_with_generate=True,
    adafactor = True,
    #fp16=True,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=student)


In [35]:
from transformers import Adafactor
#optimizer = Adafactor(student.parameters(), scale_parameter=True, relative_step=True, warmup_init=True, lr=None)

trainer = MyTrainer(
    student,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    #optimizers= (optimizer,_)
)

In [ ]:
trainer.train()

wandb: Currently logged in as: hadywalied (use `wandb login --relogin` to force relogin)


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:562: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1005.)
  exp_avg_sq_row.mul_(beta2t).add_(1.0 - beta2t, update.mean(dim=-1))


Step,Training Loss


In [ ]:
trainer.evaluate()

In [ ]:
trainer.predict(tokenized_datasets["test"])

In [ ]:
trainer.train()

In [ ]:
student.config

In [ ]:
student.config.max_length = 1024
student.config.max_position_embeddings = 1024
